In [1]:
!pip install tensorflow
!pip install transformers
!pip install pandas
!pip install scikit-learn

In [5]:
import os
import pandas as pd

file_path = './Flipkart/flipkart_com-ecommerce_sample_1050.csv'

if os.path.exists(file_path):
    data = pd.read_csv(file_path)
    print("CSV chargé avec succès.")
else:
    print(f"Erreur : Le fichier n'existe pas à l'emplacement {file_path}")


CSV chargé avec succès.


In [6]:
# Ajoutez le chemin des images et préparez les labels de catégories
data['image_path'] = data['image'].apply(lambda x: f'./Flipkart/Images/{x}')
data['category'] = data['product_category_tree'].apply(lambda x: x.split(">>")[0].replace("[", "").replace("]", "").replace('"', '').strip())
categories = data['category'].unique().tolist()
category_to_index = {cat: idx for idx, cat in enumerate(categories)}
data['category_id'] = data['category'].map(category_to_index)

In [7]:
# Séparation des données
from sklearn.model_selection import train_test_split

X_text = data['description'].fillna("")
X_image = data['image_path']
y = data['category_id']
X_train_text, X_test_text, X_train_image, X_test_image, y_train, y_test = train_test_split(X_text, X_image, y, test_size=0.2, random_state=42)

In [8]:
from transformers import TFDistilBertModel, DistilBertTokenizer
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

# Charger le tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Charger le modèle DistilBERT sans tête de classification
base_model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")

# Ajout de la couche de classification
class TextClassificationModel(Model):
    def __init__(self, base_model, num_classes):
        super(TextClassificationModel, self).__init__()
        self.base_model = base_model
        self.classifier = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        outputs = self.base_model(inputs)
        # Prendre la moyenne de last_hidden_state
        pooled_output = tf.reduce_mean(outputs.last_hidden_state, axis=1)
        return self.classifier(pooled_output)

# Initialiser le modèle de classification de texte
text_model = TextClassificationModel(base_model, num_classes=len(categories))

# Compiler le modèle
text_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), 
                   loss="sparse_categorical_crossentropy", 
                   metrics=["accuracy"])

/home/utilisateur/Bureau/week2/multi-inputs/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-29 11:52:36.725571: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-29 11:52:36.879726: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-29 11:52:37.020200: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730199157.147661  245099 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730199157.186342  24509

RuntimeError: Failed to import transformers.models.distilbert.modeling_tf_distilbert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
import numpy as np  # Importation de numpy
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import tensorflow as tf

# Prétraitement des images
def load_and_preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image

# Chargement des images
X_train_image_processed = np.array([load_and_preprocess_image(img_path) for img_path in X_train_image])
X_test_image_processed = np.array([load_and_preprocess_image(img_path) for img_path in X_test_image])

# Modèle de classification d'image
base_model = EfficientNetB0(include_top=False, input_shape=(224, 224, 3), weights="imagenet")
x = GlobalAveragePooling2D()(base_model.output)
output = Dense(len(categories), activation="softmax")(x)
image_model = Model(inputs=base_model.input, outputs=output)

image_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Entraînement du modèle
image_model.fit(X_train_image_processed, y_train, batch_size=16, epochs=3, validation_data=(X_test_image_processed, y_test))

In [ ]:
from tensorflow.keras.layers import Input, Concatenate, Dense
from tensorflow.keras.models import Model
from transformers import TFDistilBertModel
from tensorflow.keras.applications import EfficientNetB0

# Entrée pour le texte
text_input = Input(shape=(None,), dtype=tf.int32, name="input_ids")
text_mask = Input(shape=(None,), dtype=tf.int32, name="attention_mask")
text_model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")
text_features = text_model(text_input, attention_mask=text_mask).last_hidden_state[:, 0, :]

# Entrée pour l'image
image_input = Input(shape=(224, 224, 3), name="image_input")
base_image_model = EfficientNetB0(include_top=False, input_shape=(224, 224, 3), weights="imagenet")
image_features = GlobalAveragePooling2D()(base_image_model(image_input))

# Fusion des deux sorties
combined_features = Concatenate()([text_features, image_features])
output = Dense(len(categories), activation="softmax")(combined_features)

# Modèle multi-input
multi_input_model = Model(inputs=[text_input, text_mask, image_input], outputs=output)

# Compilation du modèle
multi_input_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Entraînement du modèle
multi_input_model.fit(
    [train_encodings["input_ids"], train_encodings["attention_mask"], X_train_image_processed],
    y_train,
    batch_size=16,
    epochs=3,
    validation_data=([test_encodings["input_ids"], test_encodings["attention_mask"], X_test_image_processed], y_test)
)

In [ ]:
# Préparer les données d'entrée
def predict_text_category(description):
    inputs = tokenizer(description, return_tensors="tf", padding="max_length", truncation=True)
    outputs = text_model(inputs)
    prediction = tf.argmax(outputs, axis=1).numpy()[0]
    return categories[prediction]

# Exemple de prédiction
description_test = "A powerful all-in-one computer with last generation components"
print("Predicted Category:", predict_text_category(description_test))

In [ ]:
import requests
from PIL import Image
import tensorflow as tf
import numpy as np

def load_and_preprocess_image_from_url(url):
    # Télécharger l'image depuis l'URL
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    # Ouvrir l'image avec PIL et la convertir au format attendu par le modèle
    image = Image.open(response.raw).convert("RGB")
    image = image.resize((224, 224))  # Redimensionner l'image
    image = np.array(image) / 255.0   # Normaliser les pixels entre 0 et 1
    return image

def predict_image_category_from_url(url):
    # Charger et prétraiter l'image depuis l'URL
    image = load_and_preprocess_image_from_url(url)
    image = tf.expand_dims(image, axis=0)  # Ajouter une dimension pour le batch

    # Prédire la catégorie
    prediction = tf.argmax(image_model.predict(image), axis=1).numpy()[0]
    return categories[prediction]

# URL de l'image
image_url = "https://cdn.pixabay.com/photo/2020/10/21/18/07/laptop-5673901_1280.jpg"
print("Predicted Category:", predict_image_category_from_url(image_url))

In [ ]:
import requests
from PIL import Image
import tensorflow as tf
import numpy as np
from transformers import DistilBertTokenizer

# Charger le tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def load_and_preprocess_image_from_url(url):
    # Télécharger l'image depuis l'URL
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    # Ouvrir l'image avec PIL et la convertir au format attendu par le modèle
    image = Image.open(response.raw).convert("RGB")
    image = image.resize((224, 224))  # Redimensionner l'image
    image = np.array(image) / 255.0   # Normaliser les pixels entre 0 et 1
    return image

def predict_multi_input(description, image_url):
    # Prétraitement de la description
    text_inputs = tokenizer(description, return_tensors="tf", padding="max_length", truncation=True)
    
    # Charger et prétraiter l'image depuis l'URL
    image = load_and_preprocess_image_from_url(image_url)
    image = tf.expand_dims(image, axis=0)  # Ajouter une dimension pour le batch

    # Prédire la catégorie
    prediction = tf.argmax(
        multi_input_model.predict(
            {"input_ids": text_inputs["input_ids"], "attention_mask": text_inputs["attention_mask"], "image_input": image}
        ), axis=1
    ).numpy()[0]
    return categories[prediction]

# Exemple d'URL de l'image et description
description_test = "TV support table wood style rétro"
image_url = "https://cdn1.hellin.fr/26695-zoom_default/meuble-tv-retro-en-bois-2-niches-2-portes-l180-mallet.jpg"
print("Predicted Category:", predict_multi_input(description_test, image_url))



In [ ]:
# Sauvegarde des poids du modèle de texte
text_model.save_weights('/content/text_model_weights')

# Sauvegarde des poids du modèle d'image
image_model.save_weights('/content/image_model_weights')

# Sauvegarde des poids du modèle multi-input
multi_input_model.save_weights('/content/multi_input_model_weights')

In [ ]:
import tensorflow as tf
from transformers import TFDistilBertModel, DistilBertTokenizer
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Concatenate
from tensorflow.keras.models import Model

# Recréer la structure du modèle de texte
class TextClassificationModel(Model):
    def __init__(self, base_model, num_classes):
        super(TextClassificationModel, self).__init__()
        self.base_model = base_model
        self.classifier = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        outputs = self.base_model(inputs)
        pooled_output = tf.reduce_mean(outputs.last_hidden_state, axis=1)
        return self.classifier(pooled_output)

# Instancier et charger les poids
base_model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")
text_model = TextClassificationModel(base_model, num_classes=len(categories))
text_model.load_weights('path/to/text_model_weights')

# Procédez de même pour le modèle d'image et le modèle multi-input
# Exemple pour le modèle d'image
image_model = ...  # recréez la structure du modèle d'image
image_model.load_weights('path/to/image_model_weights')

# Exemple pour le modèle multi-input
multi_input_model = ...  # recréez la structure du modèle multi-input
multi_input_model.load_weights('path/to/multi_input_model_weights')
